In [1]:
# Installing dotenv library to manage confidential keys saved as environment variables
#!pip install python-dotenv

In [2]:
import json
import pandas as pd
import re
import datetime as dt

import boto3

import os
from urllib.request import urlopen
from dotenv import load_dotenv

load_dotenv()

True

In [3]:
# Loading .json file with hotel urls
url = "https://kayak-booking-bucket-12-12-2022.s3.eu-west-3.amazonaws.com/booking_search_page.json"
response_hotel_urls = urlopen(url)
hotel_urls = json.loads(response_hotel_urls.read())

In [4]:
# Creating a dataframe from .json file
hotel_urls_df = pd.DataFrame.from_records(hotel_urls)

display(hotel_urls_df.head())

# Checking the length of the dataframe
print("Length of the dataframe:", len(hotel_urls_df))

,hotel_name,hotel_url,booking_city_url
0,Lagrange Vacances Les Chalets d’Ax,https://www.booking.com/hotel/fr/residence-les...,<HtmlResponse 200 https://www.booking.com/sear...
1,GuestReady - Cozy perfection in the city centre,https://www.booking.com/hotel/fr/guestready-to...,<HtmlResponse 200 https://www.booking.com/sear...
2,Les gîtes de Beille,https://www.booking.com/hotel/fr/les-gites-de-...,<HtmlResponse 200 https://www.booking.com/sear...
3,Le petit nid,https://www.booking.com/hotel/fr/le-petit-nid-...,<HtmlResponse 200 https://www.booking.com/sear...
4,Appartement Fontargente,https://www.booking.com/hotel/fr/appartement-f...,<HtmlResponse 200 https://www.booking.com/sear...


Length of the dataframe: 1000


The "booking_city_url" column contains the url that was used to obtain a list of hotels in a given city. In order to know the city where each of the hotels is situated, let us extract the name of the city from the url.

In [5]:
# Example of text in column "booking_city_url"
print(hotel_urls_df.iloc[0]["booking_city_url"])

# Splitting the url using "=" and "&" as delimiters and conserving part of url as the name of the city
hotel_urls_df["city_name"] = hotel_urls_df["booking_city_url"].apply(lambda x: re.split(r"=|&", x)[1])

# Replacing "+" with " " in the names of cities consisting of several words
hotel_urls_df["city_name"] = hotel_urls_df["city_name"].apply(lambda x: x.replace("+", " "))

# Dropping "booking_city_url" column
hotel_urls_df = hotel_urls_df.drop("booking_city_url", axis=1)

hotel_urls_df.head()

<HtmlResponse 200 https://www.booking.com/searchresults.en-us.html?ss=Ariege&ssne=Ariege&ssne_untouched=Ariege&order=distance_from_search>


,hotel_name,hotel_url,city_name
0,Lagrange Vacances Les Chalets d’Ax,https://www.booking.com/hotel/fr/residence-les...,Ariege
1,GuestReady - Cozy perfection in the city centre,https://www.booking.com/hotel/fr/guestready-to...,Annecy
2,Les gîtes de Beille,https://www.booking.com/hotel/fr/les-gites-de-...,Ariege
3,Le petit nid,https://www.booking.com/hotel/fr/le-petit-nid-...,Ariege
4,Appartement Fontargente,https://www.booking.com/hotel/fr/appartement-f...,Ariege


We will now use join the table containing hotel urls with the table containing other data for each hotel (score, address, description etc).

In [6]:
# Loading .json file with hotels data
url = "https://kayak-booking-bucket-12-12-2022.s3.eu-west-3.amazonaws.com/booking_hotels_data.json"
response_hotels_data = urlopen(url)
hotels = json.loads(response_hotels_data.read())

In [7]:
# Creating a dataframe from .json file
hotels_df = pd.DataFrame.from_records(hotels)

hotels_df.head()

,hotel_name,score,description,location,latitide,longtitude,hotel_url
0,GuestReady - Cozy perfection in the city centre,8.0,"Located in the centre of Annecy, 37 km from Ro...","\n8 Rue Camille Dunant, 74000 Annecy, France\n",45.89994070,6.12705460,<HtmlResponse 200 https://www.booking.com/hote...
1,Appartement Fontargente,9.2,Appartement Fontargente is located in Ax-les-T...,\npremier étage 19 Avenue Docteur François Gom...,42.71847547,1.84064515,<HtmlResponse 200 https://www.booking.com/hote...
2,Résidence Néméa Les Balcons d'Ax,7.1,Résidence Néméa Les Balcons d'Ax is located in...,"\nStation De Bonascre, 09110 Ax-les-Thermes, F...",42.70237131,1.81530745,<HtmlResponse 200 https://www.booking.com/hote...
3,Lagrange Vacances Les Chalets d’Ax,7.6,You're eligible for a Genius discount at Lagra...,\nQuartier De Castel Maou - Chemin d'Aouredou ...,42.71383156,1.83920681,<HtmlResponse 200 https://www.booking.com/hote...
4,Terres de France - Domaine du Palais,7.5,Located in Saint-Lizier in the Midi-Pyrénées r...,"\nChemin du Parc Le Palais des Evêques, 09190 ...",43.00305100,1.13713000,<HtmlResponse 200 https://www.booking.com/hote...


In [8]:
# Renaming columns for clarity
hotels_df.rename(columns = {'latitide':'hotel_latitude'}, inplace = True)
hotels_df.rename(columns = {'longtitude':'hotel_longtitude'}, inplace = True)

# Addresses of hotels contain html tag /n, we will use .strip() in  order to get rid of it
hotels_df["location"] = hotels_df["location"].apply(lambda x: str.strip(x))

# Checking the resulting dataframe
hotels_df.head()

,hotel_name,score,description,location,hotel_latitude,hotel_longtitude,hotel_url
0,GuestReady - Cozy perfection in the city centre,8.0,"Located in the centre of Annecy, 37 km from Ro...","8 Rue Camille Dunant, 74000 Annecy, France",45.89994070,6.12705460,<HtmlResponse 200 https://www.booking.com/hote...
1,Appartement Fontargente,9.2,Appartement Fontargente is located in Ax-les-T...,premier étage 19 Avenue Docteur François Gomma...,42.71847547,1.84064515,<HtmlResponse 200 https://www.booking.com/hote...
2,Résidence Néméa Les Balcons d'Ax,7.1,Résidence Néméa Les Balcons d'Ax is located in...,"Station De Bonascre, 09110 Ax-les-Thermes, France",42.70237131,1.81530745,<HtmlResponse 200 https://www.booking.com/hote...
3,Lagrange Vacances Les Chalets d’Ax,7.6,You're eligible for a Genius discount at Lagra...,"Quartier De Castel Maou - Chemin d'Aouredou -,...",42.71383156,1.83920681,<HtmlResponse 200 https://www.booking.com/hote...
4,Terres de France - Domaine du Palais,7.5,Located in Saint-Lizier in the Midi-Pyrénées r...,"Chemin du Parc Le Palais des Evêques, 09190 Sa...",43.00305100,1.13713000,<HtmlResponse 200 https://www.booking.com/hote...


Some hotel descriptions begin with promotional information about a discount. We will remove these phrases from descriptions.

In [9]:
# Example of hotel description with promotional information
print("Description before cleaning:")
print("'", hotels_df.iloc[3]["description"], "'")
print()

# If the description contains words "Genius discount", we only keep the text the follows the phrase ""To save at this property, all you have to do is  ."
hotels_df["description"] = hotels_df["description"].apply(lambda x: x.split("To save at this property, all you have to do is  . ")[1] if 'Genius discount' in x else x)

print("Description after cleaning: ")
print("'", hotels_df.iloc[3]["description"], "'")

Description before cleaning:
You're eligible for a Genius discount at Lagrange Vacances Les Chalets d’Ax! To save at this property, all you have to do is  . Lagrange Vacances Les Chalets d’Ax is located in Ax-Les-Thermes, 800 m from the town centre and ski lifts. It offers WiFi in the apartments at an extra cost and an outdoor seasonal heated swimming pool. Lagrange Vacances Les Chalets d’Ax provides studios and apartments with a private balcony or terrace. They each have a kitchen equipped with a dishwasher and microwave. Local leisure activities include fishing in the Ariege River, cycling and hiking. In winter, during holidays and week-ends, free shuttles run from the property to Ax-les-Thermes ski lift. Tarascon Prehistoric Art Park and the Niaux caves can be visited a 30-minute drive from Lagrange Vacances Les Chalets d’Ax which provides free private parking. 

Description after cleaning: 
Lagrange Vacances Les Chalets d’Ax is located in Ax-Les-Thermes, 800 m from the town centre 

In [11]:
# Removing part of text from "hotel_url" column and keeping only the cleaned url
hotels_df["hotel_url"] = hotels_df["hotel_url"].apply(lambda x: x.strip("<HtmlResponse 200 ").split("?")[0])

# Some hotel urls contain "en_gb." part indicating that the url corresponds to the English-language version of the page. 
print("Example of url:", hotels_df.iloc[5]["hotel_url"])

# The hotel url remains functional without the part that indicates the language version of the page,
# so we'll remove this part of string for uniformity (the url remains functional).
hotels_df["hotel_url"] = hotels_df["hotel_url"].apply(lambda x: x.replace("en-gb.", "") if "en-gb." in x else x)

hotels_df.head()

Example of url: https://www.booking.com/hotel/fr/chalet-bois-au-milieu-des-pyrenees.en-gb.html


,hotel_name,score,description,location,hotel_latitude,hotel_longtitude,hotel_url
0,GuestReady - Cozy perfection in the city centre,8.0,"Located in the centre of Annecy, 37 km from Ro...","8 Rue Camille Dunant, 74000 Annecy, France",45.89994070,6.12705460,https://www.booking.com/hotel/fr/guestready-to...
1,Appartement Fontargente,9.2,Appartement Fontargente is located in Ax-les-T...,premier étage 19 Avenue Docteur François Gomma...,42.71847547,1.84064515,https://www.booking.com/hotel/fr/appartement-f...
2,Résidence Néméa Les Balcons d'Ax,7.1,Résidence Néméa Les Balcons d'Ax is located in...,"Station De Bonascre, 09110 Ax-les-Thermes, France",42.70237131,1.81530745,https://www.booking.com/hotel/fr/residence-les...
3,Lagrange Vacances Les Chalets d’Ax,7.6,Lagrange Vacances Les Chalets d’Ax is located ...,"Quartier De Castel Maou - Chemin d'Aouredou -,...",42.71383156,1.83920681,https://www.booking.com/hotel/fr/residence-les...
4,Terres de France - Domaine du Palais,7.5,Located in Saint-Lizier in the Midi-Pyrénées r...,"Chemin du Parc Le Palais des Evêques, 09190 Sa...",43.00305100,1.13713000,https://www.booking.com/hotel/fr/terres-de-fra...


Merging the table containing hotel urls and the table containing other information about hotels

In [12]:
# Rearranging order of columns to prepare the table for the merge with hotel_urls_df
hotels_df = hotels_df[['hotel_url', 'hotel_name', 'score', 'description', 'location', 'hotel_latitude',
       'hotel_longtitude']]

# Merging tables
hotel_info_df = hotel_urls_df.merge(hotels_df, how="left")

# Checking the resulting table
hotel_info_df.head()

,hotel_name,hotel_url,city_name,score,description,location,hotel_latitude,hotel_longtitude
0,Lagrange Vacances Les Chalets d’Ax,https://www.booking.com/hotel/fr/residence-les...,Ariege,7.6,Lagrange Vacances Les Chalets d’Ax is located ...,"Quartier De Castel Maou - Chemin d'Aouredou -,...",42.71383156,1.83920681
1,GuestReady - Cozy perfection in the city centre,https://www.booking.com/hotel/fr/guestready-to...,Annecy,8.0,"Located in the centre of Annecy, 37 km from Ro...","8 Rue Camille Dunant, 74000 Annecy, France",45.89994070,6.12705460
2,Les gîtes de Beille,https://www.booking.com/hotel/fr/les-gites-de-...,Ariege,8.1,Located in Les Cabannes in the Midi-Pyrénées r...,"25 Quartier la Bexane, 09310 Les Cabannes, France",42.78555710,1.68166541
3,Le petit nid,https://www.booking.com/hotel/fr/le-petit-nid-...,Ariege,9.2,"Offering garden views, Le petit nid is an acco...","RDC 1 Rue de la Place, 09330 Montgaillard, France",42.93227802,1.63516787
4,Appartement Fontargente,https://www.booking.com/hotel/fr/appartement-f...,Ariege,9.2,Appartement Fontargente is located in Ax-les-T...,premier étage 19 Avenue Docteur François Gomma...,42.71847547,1.84064515


In [13]:
# Checking if there are any missing values in the table
hotel_info_df.isnull().sum()

hotel_name           0
hotel_url            0
city_name            0
score               95
description          0
location             0
hotel_latitude       0
hotel_longtitude     0
dtype: int64

It turns out that there are a number of hotels with missing scores. Let's have a closer look at them to see if there's a problem.

In [88]:
display(hotel_info_df[hotel_info_df["score"].isnull()].head())

# Saving indexes of rows where the hotel score is missing
no_score_indexes = hotel_info_df[hotel_info_df["score"].isnull()].index

,hotel_name,hotel_url,city_name,score,description,location,hotel_latitude,hotel_longtitude
41,Le Factory : Duplex en vieille ville côté lac,https://www.booking.com/hotel/fr/le-factory-du...,Annecy,None,"Offering free WiFi, Le Factory: Duplex en viei...","9 Rue Grenette, 74000 Annecy, France",45.89910140,6.12708550
48,Appartement de 2 chambres avec wifi a Annecy,https://www.booking.com/hotel/fr/apartment-rue...,Annecy,None,"Located in the centre of Annecy, 37 km from Ro...","23 Rue Notre Dame Haute-Savoie, 74000 Annecy, ...",45.89997080,6.12629850
56,Le Saboly,https://www.booking.com/hotel/fr/le-saboly-avi...,Avignon,None,Situated less than 1 km from Avignon Central S...,"2B Place Nicolas Saboly, 84000 Avignon, France",43.94877820,4.80732660
62,"Les Logis de Halley, au cœur de la cité des Papes",https://www.booking.com/hotel/fr/au-coeur-de-l...,Avignon,None,"Set in the centre of Avignon, just 400 m from ...","2ème étage 2 Rue Edmond Halley, 84000 Avignon,...",43.94883313,4.80700983
66,Loft hyper centre,https://www.booking.com/hotel/fr/loft-hyper-ce...,Avignon,None,"Situated in the centre of Avignon, just 400 m ...","4 Rue de la Rappe, 84000 Avignon, France",43.94848300,4.80745830


If we print the descriptions of hotels that have no score, it turns out that most of these are in fact apartments or villas.
If they have been present on booking.com not for a long time, it is normal that they do not have a score yet as there are many less visitors than in hotels.

In [93]:
# (code below commented to avoid long output)
"""for index in no_score_indexes:
    print(hotel_info_df["description"].iloc[index])
    print()"""

'for index in no_score_indexes:\n    print(hotel_info_df["description"].iloc[index])\n    print()'

Enriching the table with city coordinates

In [22]:
# Loading .csv file with city coordinates from S3 bucket
url = 'https://kayak-booking-bucket-12-12-2022.s3.eu-west-3.amazonaws.com/city_coordinates.csv'
city_coord = pd.read_csv(url)
city_coord.head(5)

,place_id,city_name,lat,lon
0,156094680,Mont Saint-Michel,48.635954,-1.511460
1,297756747,Saint-Malo,48.649518,-2.026041
2,297981358,Bayeux,49.276462,-0.702474
3,298137491,Le Havre,49.493898,0.107973
4,297518815,Rouen,49.440459,1.093966


In [23]:
city_coord["city_name"].unique()

array(['Mont Saint-Michel', 'Saint-Malo', 'Bayeux', 'Le Havre', 'Rouen',
       'Paris', 'Amiens', 'Lille', 'Strasbourg',
       'Château du Haut-Kœnigsbourg', 'Colmar', 'Eguisheim', 'Besançon',
       'Dijon', 'Annecy', 'Grenoble', 'Lyon', 'Gorges du Verdon',
       'Bormes-les-Mimosas', 'Cassis', 'Marseilla', 'Aix-en-Provence',
       'Avignon', 'Uzès', 'Nîmes', 'Aigues-Mortes',
       'Saintes-Maries-de-la-Mer', 'Collioure', 'Carcassonne', 'Ariège',
       'Toulouse', 'Montauban', 'Biarritz', 'Bayonne', 'La Rochelle'],
      dtype=object)

In [24]:
len(city_coord["city_name"].unique())

35

In [25]:
city_names_coord= city_coord["city_name"].tolist()

In [26]:
city_names_coord.sort()

In [27]:
len(city_names_coord)

35

In [28]:
hotel_info_df["city_name"].unique().tolist()

['Ariege',
 'Annecy',
 'Avignon',
 'Aigues Mortes',
 'Bayeux',
 'Aix en Provence',
 'Bayonne',
 'Amiens',
 'Bormes les Mimosas',
 'Besancon',
 'Carcassonne',
 'Chateau du Haut Koenigsbourg',
 'Biarritz',
 'Cassis',
 'Collioure',
 'Colmar',
 'Eguisheim',
 'Grenoble',
 'Dijon',
 'Gorges du Verdon',
 'La Rochelle',
 'Lyon',
 'Lille',
 'Le Havre',
 'Marseille',
 'Montauban',
 'Nimes',
 'Mont Saint Michel',
 'Paris',
 'Rouen',
 'Strasbourg',
 'St Malo',
 'Saintes Maries de la mer',
 'Uzes',
 'Toulouse']

In [29]:
city_names_hotel = hotel_info_df["city_name"].unique().tolist()

In [30]:
len(city_names_hotel)

35

In [31]:
city_names_hotel.sort()

In [32]:
len(city_names_hotel)

35

In [33]:
print(city_names_coord)

print(city_names_hotel)

['Aigues-Mortes', 'Aix-en-Provence', 'Amiens', 'Annecy', 'Ariège', 'Avignon', 'Bayeux', 'Bayonne', 'Besançon', 'Biarritz', 'Bormes-les-Mimosas', 'Carcassonne', 'Cassis', 'Château du Haut-Kœnigsbourg', 'Collioure', 'Colmar', 'Dijon', 'Eguisheim', 'Gorges du Verdon', 'Grenoble', 'La Rochelle', 'Le Havre', 'Lille', 'Lyon', 'Marseilla', 'Mont Saint-Michel', 'Montauban', 'Nîmes', 'Paris', 'Rouen', 'Saint-Malo', 'Saintes-Maries-de-la-Mer', 'Strasbourg', 'Toulouse', 'Uzès']
['Aigues Mortes', 'Aix en Provence', 'Amiens', 'Annecy', 'Ariege', 'Avignon', 'Bayeux', 'Bayonne', 'Besancon', 'Biarritz', 'Bormes les Mimosas', 'Carcassonne', 'Cassis', 'Chateau du Haut Koenigsbourg', 'Collioure', 'Colmar', 'Dijon', 'Eguisheim', 'Gorges du Verdon', 'Grenoble', 'La Rochelle', 'Le Havre', 'Lille', 'Lyon', 'Marseille', 'Mont Saint Michel', 'Montauban', 'Nimes', 'Paris', 'Rouen', 'Saintes Maries de la mer', 'St Malo', 'Strasbourg', 'Toulouse', 'Uzes']


In [34]:
city_names_coord = [city.replace("Marseilla", "Marseille") for city in city_names_coord]


In [35]:
print(city_names_coord)

print(city_names_hotel)

['Aigues-Mortes', 'Aix-en-Provence', 'Amiens', 'Annecy', 'Ariège', 'Avignon', 'Bayeux', 'Bayonne', 'Besançon', 'Biarritz', 'Bormes-les-Mimosas', 'Carcassonne', 'Cassis', 'Château du Haut-Kœnigsbourg', 'Collioure', 'Colmar', 'Dijon', 'Eguisheim', 'Gorges du Verdon', 'Grenoble', 'La Rochelle', 'Le Havre', 'Lille', 'Lyon', 'Marseille', 'Mont Saint-Michel', 'Montauban', 'Nîmes', 'Paris', 'Rouen', 'Saint-Malo', 'Saintes-Maries-de-la-Mer', 'Strasbourg', 'Toulouse', 'Uzès']
['Aigues Mortes', 'Aix en Provence', 'Amiens', 'Annecy', 'Ariege', 'Avignon', 'Bayeux', 'Bayonne', 'Besancon', 'Biarritz', 'Bormes les Mimosas', 'Carcassonne', 'Cassis', 'Chateau du Haut Koenigsbourg', 'Collioure', 'Colmar', 'Dijon', 'Eguisheim', 'Gorges du Verdon', 'Grenoble', 'La Rochelle', 'Le Havre', 'Lille', 'Lyon', 'Marseille', 'Mont Saint Michel', 'Montauban', 'Nimes', 'Paris', 'Rouen', 'Saintes Maries de la mer', 'St Malo', 'Strasbourg', 'Toulouse', 'Uzes']


In [36]:
print(list(zip(city_names_hotel, city_names_coord)))

[('Aigues Mortes', 'Aigues-Mortes'), ('Aix en Provence', 'Aix-en-Provence'), ('Amiens', 'Amiens'), ('Annecy', 'Annecy'), ('Ariege', 'Ariège'), ('Avignon', 'Avignon'), ('Bayeux', 'Bayeux'), ('Bayonne', 'Bayonne'), ('Besancon', 'Besançon'), ('Biarritz', 'Biarritz'), ('Bormes les Mimosas', 'Bormes-les-Mimosas'), ('Carcassonne', 'Carcassonne'), ('Cassis', 'Cassis'), ('Chateau du Haut Koenigsbourg', 'Château du Haut-Kœnigsbourg'), ('Collioure', 'Collioure'), ('Colmar', 'Colmar'), ('Dijon', 'Dijon'), ('Eguisheim', 'Eguisheim'), ('Gorges du Verdon', 'Gorges du Verdon'), ('Grenoble', 'Grenoble'), ('La Rochelle', 'La Rochelle'), ('Le Havre', 'Le Havre'), ('Lille', 'Lille'), ('Lyon', 'Lyon'), ('Marseille', 'Marseille'), ('Mont Saint Michel', 'Mont Saint-Michel'), ('Montauban', 'Montauban'), ('Nimes', 'Nîmes'), ('Paris', 'Paris'), ('Rouen', 'Rouen'), ('Saintes Maries de la mer', 'Saint-Malo'), ('St Malo', 'Saintes-Maries-de-la-Mer'), ('Strasbourg', 'Strasbourg'), ('Toulouse', 'Toulouse'), ('Uzes'

In [37]:
"""for i in range(0, len(hotel_info_df)):
    if hotel_info_df["city_name"].loc[i] in city_names:
        index = city_names.index(hotel_info_df["city_name"].loc[i])
        hotel_info_df["city_name"].loc[i].replace(hotel_info_df["city_name"].loc[i], city_list[index])
    else:
        hotel_info_df["city_name"].loc[i].replace(hotel_info_df["city_name"].loc[i], "None")"""

'for i in range(0, len(hotel_info_df)):\n    if hotel_info_df["city_name"].loc[i] in city_names:\n        index = city_names.index(hotel_info_df["city_name"].loc[i])\n        hotel_info_df["city_name"].loc[i].replace(hotel_info_df["city_name"].loc[i], city_list[index])\n    else:\n        hotel_info_df["city_name"].loc[i].replace(hotel_info_df["city_name"].loc[i], "None")'

In [38]:
for i in range(0, len(hotel_info_df["city_name"])):
    if hotel_info_df["city_name"][i] in city_names_hotel:
        index = city_names_hotel.index(hotel_info_df["city_name"].loc[i])
        hotel_info_df["city_name"] = hotel_info_df["city_name"].replace(hotel_info_df["city_name"][i], city_names_coord[index])
    else: 
        pass
        

In [39]:
hotel_info_df.head()

,hotel_name,hotel_url,city_name,score,description,location,hotel_latitude,hotel_longtitude
0,Lagrange Vacances Les Chalets d’Ax,https://www.booking.com/hotel/fr/residence-les...,Ariège,7.6,Lagrange Vacances Les Chalets d’Ax is located ...,"Quartier De Castel Maou - Chemin d'Aouredou -,...",42.71383156,1.83920681
1,GuestReady - Cozy perfection in the city centre,https://www.booking.com/hotel/fr/guestready-to...,Annecy,8.0,"Located in the centre of Annecy, 37 km from Ro...","8 Rue Camille Dunant, 74000 Annecy, France",45.89994070,6.12705460
2,Les gîtes de Beille,https://www.booking.com/hotel/fr/les-gites-de-...,Ariège,8.1,Located in Les Cabannes in the Midi-Pyrénées r...,"25 Quartier la Bexane, 09310 Les Cabannes, France",42.78555710,1.68166541
3,Le petit nid,https://www.booking.com/hotel/fr/le-petit-nid-...,Ariège,9.2,"Offering garden views, Le petit nid is an acco...","RDC 1 Rue de la Place, 09330 Montgaillard, France",42.93227802,1.63516787
4,Appartement Fontargente,https://www.booking.com/hotel/fr/appartement-f...,Ariège,9.2,Appartement Fontargente is located in Ax-les-T...,premier étage 19 Avenue Docteur François Gomma...,42.71847547,1.84064515


In [40]:
hotel_info_df.tail()

,hotel_name,hotel_url,city_name,score,description,location,hotel_latitude,hotel_longtitude
995,Les Toits du Capitole - Loft de standing 65 m2...,https://www.booking.com/hotel/fr/les-toits-du-...,Toulouse,9.2,Les Toits du Capitole is a spacious and modern...,"6, rue des Lois, 31000 Toulouse, France",43.60510201,1.44236863
996,La Parenthèse cosy - Appartement climatisé en ...,https://www.booking.com/hotel/fr/le-lafayette-...,Toulouse,8.3,"Ideally situated in the centre of Toulouse, La...","20 Rue Lafayette, 31000 Toulouse, France",43.60484808,1.44617052
997,T2 COSY - CAPITOLE - CALME - Wi-Fi - NETFLIX,https://www.booking.com/hotel/fr/studio-cosy-h...,Toulouse,8.7,"Situated in the centre of Toulouse, 3.7 km fro...","1er étage 44 Rue Saint-Rome, 31000 Toulouse, F...",43.60330820,1.44347870
998,Modern flat 50m from the Capitole - Toulouse -...,https://www.booking.com/hotel/fr/modern-flat-5...,Toulouse,8.3,"Set in the centre of Toulouse, 3.3 km from Zén...","43 rue Saint-Rome, 31000 Toulouse, France",43.60333730,1.44330860
999,Hotel Albert 1er,https://www.booking.com/hotel/fr/hotelalbertto...,Toulouse,8.3,"Located in the heart of Toulouse, the Hotel Al...","8 rue Rivals, 31000 Toulouse, France",43.60603220,1.44491941


In [41]:
hotel_info_df["city_name"].nunique()

35

In [42]:
len(city_names_coord)

35

In [43]:
city_coord.columns

Index(['place_id', 'city_name', 'lat', 'lon'], dtype='object')

In [44]:
city_coord = city_coord[['place_id',  'lat', 'lon', 'city_name']]

In [45]:
city_coord.head()

,place_id,lat,lon,city_name
0,156094680,48.635954,-1.511460,Mont Saint-Michel
1,297756747,48.649518,-2.026041,Saint-Malo
2,297981358,49.276462,-0.702474,Bayeux
3,298137491,49.493898,0.107973,Le Havre
4,297518815,49.440459,1.093966,Rouen


In [46]:
city_and_hotel_info_df = city_coord.merge(hotel_info_df, how="right")

In [47]:
city_and_hotel_info_df.columns

Index(['place_id', 'lat', 'lon', 'city_name', 'hotel_name', 'hotel_url',
       'score', 'description', 'location', 'hotel_latitude',
       'hotel_longtitude'],
      dtype='object')

In [48]:
city_and_hotel_info_df = city_and_hotel_info_df[['city_name', 'place_id', 'lat', 'lon', 'hotel_name', 'hotel_url',
       'score', 'description', 'location', 'hotel_latitude', 'hotel_longtitude']]

In [49]:
city_and_hotel_info_df.head()

,city_name,place_id,lat,lon,hotel_name,hotel_url,score,description,location,hotel_latitude,hotel_longtitude
0,Ariège,297389050.0,42.945537,1.406554,Lagrange Vacances Les Chalets d’Ax,https://www.booking.com/hotel/fr/residence-les...,7.6,Lagrange Vacances Les Chalets d’Ax is located ...,"Quartier De Castel Maou - Chemin d'Aouredou -,...",42.71383156,1.83920681
1,Annecy,298516920.0,45.899235,6.128885,GuestReady - Cozy perfection in the city centre,https://www.booking.com/hotel/fr/guestready-to...,8.0,"Located in the centre of Annecy, 37 km from Ro...","8 Rue Camille Dunant, 74000 Annecy, France",45.89994070,6.12705460
2,Ariège,297389050.0,42.945537,1.406554,Les gîtes de Beille,https://www.booking.com/hotel/fr/les-gites-de-...,8.1,Located in Les Cabannes in the Midi-Pyrénées r...,"25 Quartier la Bexane, 09310 Les Cabannes, France",42.78555710,1.68166541
3,Ariège,297389050.0,42.945537,1.406554,Le petit nid,https://www.booking.com/hotel/fr/le-petit-nid-...,9.2,"Offering garden views, Le petit nid is an acco...","RDC 1 Rue de la Place, 09330 Montgaillard, France",42.93227802,1.63516787
4,Ariège,297389050.0,42.945537,1.406554,Appartement Fontargente,https://www.booking.com/hotel/fr/appartement-f...,9.2,Appartement Fontargente is located in Ax-les-T...,premier étage 19 Avenue Docteur François Gomma...,42.71847547,1.84064515


In [50]:
nb_hotels = len(city_and_hotel_info_df)

In [51]:
nb_hotels

1000

We need to add 7-day weather forecast for each hotel based on the city where the hotel is situated.

In [52]:
city_and_hotel_info_df = city_and_hotel_info_df.loc[city_and_hotel_info_df.index.repeat(7)]

In [53]:
city_and_hotel_info_df.head(10)

,city_name,place_id,lat,lon,hotel_name,hotel_url,score,description,location,hotel_latitude,hotel_longtitude
0,Ariège,297389050.0,42.945537,1.406554,Lagrange Vacances Les Chalets d’Ax,https://www.booking.com/hotel/fr/residence-les...,7.6,Lagrange Vacances Les Chalets d’Ax is located ...,"Quartier De Castel Maou - Chemin d'Aouredou -,...",42.71383156,1.83920681
0,Ariège,297389050.0,42.945537,1.406554,Lagrange Vacances Les Chalets d’Ax,https://www.booking.com/hotel/fr/residence-les...,7.6,Lagrange Vacances Les Chalets d’Ax is located ...,"Quartier De Castel Maou - Chemin d'Aouredou -,...",42.71383156,1.83920681
0,Ariège,297389050.0,42.945537,1.406554,Lagrange Vacances Les Chalets d’Ax,https://www.booking.com/hotel/fr/residence-les...,7.6,Lagrange Vacances Les Chalets d’Ax is located ...,"Quartier De Castel Maou - Chemin d'Aouredou -,...",42.71383156,1.83920681
0,Ariège,297389050.0,42.945537,1.406554,Lagrange Vacances Les Chalets d’Ax,https://www.booking.com/hotel/fr/residence-les...,7.6,Lagrange Vacances Les Chalets d’Ax is located ...,"Quartier De Castel Maou - Chemin d'Aouredou -,...",42.71383156,1.83920681
0,Ariège,297389050.0,42.945537,1.406554,Lagrange Vacances Les Chalets d’Ax,https://www.booking.com/hotel/fr/residence-les...,7.6,Lagrange Vacances Les Chalets d’Ax is located ...,"Quartier De Castel Maou - Chemin d'Aouredou -,...",42.71383156,1.83920681
0,Ariège,297389050.0,42.945537,1.406554,Lagrange Vacances Les Chalets d’Ax,https://www.booking.com/hotel/fr/residence-les...,7.6,Lagrange Vacances Les Chalets d’Ax is located ...,"Quartier De Castel Maou - Chemin d'Aouredou -,...",42.71383156,1.83920681
0,Ariège,297389050.0,42.945537,1.406554,Lagrange Vacances Les Chalets d’Ax,https://www.booking.com/hotel/fr/residence-les...,7.6,Lagrange Vacances Les Chalets d’Ax is located ...,"Quartier De Castel Maou - Chemin d'Aouredou -,...",42.71383156,1.83920681
1,Annecy,298516920.0,45.899235,6.128885,GuestReady - Cozy perfection in the city centre,https://www.booking.com/hotel/fr/guestready-to...,8.0,"Located in the centre of Annecy, 37 km from Ro...","8 Rue Camille Dunant, 74000 Annecy, France",45.89994070,6.12705460
1,Annecy,298516920.0,45.899235,6.128885,GuestReady - Cozy perfection in the city centre,https://www.booking.com/hotel/fr/guestready-to...,8.0,"Located in the centre of Annecy, 37 km from Ro...","8 Rue Camille Dunant, 74000 Annecy, France",45.89994070,6.12705460
1,Annecy,298516920.0,45.899235,6.128885,GuestReady - Cozy perfection in the city centre,https://www.booking.com/hotel/fr/guestready-to...,8.0,"Located in the centre of Annecy, 37 km from Ro...","8 Rue Camille Dunant, 74000 Annecy, France",45.89994070,6.12705460


We wil now create a key that will later help us join the dataframe with the hotel information and the dataframe with the weather information.

In [54]:
city_and_hotel_info_df.head(10)


,city_name,place_id,lat,lon,hotel_name,hotel_url,score,description,location,hotel_latitude,hotel_longtitude
0,Ariège,297389050.0,42.945537,1.406554,Lagrange Vacances Les Chalets d’Ax,https://www.booking.com/hotel/fr/residence-les...,7.6,Lagrange Vacances Les Chalets d’Ax is located ...,"Quartier De Castel Maou - Chemin d'Aouredou -,...",42.71383156,1.83920681
0,Ariège,297389050.0,42.945537,1.406554,Lagrange Vacances Les Chalets d’Ax,https://www.booking.com/hotel/fr/residence-les...,7.6,Lagrange Vacances Les Chalets d’Ax is located ...,"Quartier De Castel Maou - Chemin d'Aouredou -,...",42.71383156,1.83920681
0,Ariège,297389050.0,42.945537,1.406554,Lagrange Vacances Les Chalets d’Ax,https://www.booking.com/hotel/fr/residence-les...,7.6,Lagrange Vacances Les Chalets d’Ax is located ...,"Quartier De Castel Maou - Chemin d'Aouredou -,...",42.71383156,1.83920681
0,Ariège,297389050.0,42.945537,1.406554,Lagrange Vacances Les Chalets d’Ax,https://www.booking.com/hotel/fr/residence-les...,7.6,Lagrange Vacances Les Chalets d’Ax is located ...,"Quartier De Castel Maou - Chemin d'Aouredou -,...",42.71383156,1.83920681
0,Ariège,297389050.0,42.945537,1.406554,Lagrange Vacances Les Chalets d’Ax,https://www.booking.com/hotel/fr/residence-les...,7.6,Lagrange Vacances Les Chalets d’Ax is located ...,"Quartier De Castel Maou - Chemin d'Aouredou -,...",42.71383156,1.83920681
0,Ariège,297389050.0,42.945537,1.406554,Lagrange Vacances Les Chalets d’Ax,https://www.booking.com/hotel/fr/residence-les...,7.6,Lagrange Vacances Les Chalets d’Ax is located ...,"Quartier De Castel Maou - Chemin d'Aouredou -,...",42.71383156,1.83920681
0,Ariège,297389050.0,42.945537,1.406554,Lagrange Vacances Les Chalets d’Ax,https://www.booking.com/hotel/fr/residence-les...,7.6,Lagrange Vacances Les Chalets d’Ax is located ...,"Quartier De Castel Maou - Chemin d'Aouredou -,...",42.71383156,1.83920681
1,Annecy,298516920.0,45.899235,6.128885,GuestReady - Cozy perfection in the city centre,https://www.booking.com/hotel/fr/guestready-to...,8.0,"Located in the centre of Annecy, 37 km from Ro...","8 Rue Camille Dunant, 74000 Annecy, France",45.89994070,6.12705460
1,Annecy,298516920.0,45.899235,6.128885,GuestReady - Cozy perfection in the city centre,https://www.booking.com/hotel/fr/guestready-to...,8.0,"Located in the centre of Annecy, 37 km from Ro...","8 Rue Camille Dunant, 74000 Annecy, France",45.89994070,6.12705460
1,Annecy,298516920.0,45.899235,6.128885,GuestReady - Cozy perfection in the city centre,https://www.booking.com/hotel/fr/guestready-to...,8.0,"Located in the centre of Annecy, 37 km from Ro...","8 Rue Camille Dunant, 74000 Annecy, France",45.89994070,6.12705460


In [55]:
days = [1, 2, 3, 4, 5, 6, 7]
day_numbers = days * nb_hotels
city_and_hotel_info_df["day_in_city"] = day_numbers
city_and_hotel_info_df["day_in_city"] = city_and_hotel_info_df['city_name'] + " day "+ city_and_hotel_info_df["day_in_city"].astype(str)

    

In [56]:
city_and_hotel_info_df.head()

,city_name,place_id,lat,lon,hotel_name,hotel_url,score,description,location,hotel_latitude,hotel_longtitude,day_in_city
0,Ariège,297389050.0,42.945537,1.406554,Lagrange Vacances Les Chalets d’Ax,https://www.booking.com/hotel/fr/residence-les...,7.6,Lagrange Vacances Les Chalets d’Ax is located ...,"Quartier De Castel Maou - Chemin d'Aouredou -,...",42.71383156,1.83920681,Ariège day 1
0,Ariège,297389050.0,42.945537,1.406554,Lagrange Vacances Les Chalets d’Ax,https://www.booking.com/hotel/fr/residence-les...,7.6,Lagrange Vacances Les Chalets d’Ax is located ...,"Quartier De Castel Maou - Chemin d'Aouredou -,...",42.71383156,1.83920681,Ariège day 2
0,Ariège,297389050.0,42.945537,1.406554,Lagrange Vacances Les Chalets d’Ax,https://www.booking.com/hotel/fr/residence-les...,7.6,Lagrange Vacances Les Chalets d’Ax is located ...,"Quartier De Castel Maou - Chemin d'Aouredou -,...",42.71383156,1.83920681,Ariège day 3
0,Ariège,297389050.0,42.945537,1.406554,Lagrange Vacances Les Chalets d’Ax,https://www.booking.com/hotel/fr/residence-les...,7.6,Lagrange Vacances Les Chalets d’Ax is located ...,"Quartier De Castel Maou - Chemin d'Aouredou -,...",42.71383156,1.83920681,Ariège day 4
0,Ariège,297389050.0,42.945537,1.406554,Lagrange Vacances Les Chalets d’Ax,https://www.booking.com/hotel/fr/residence-les...,7.6,Lagrange Vacances Les Chalets d’Ax is located ...,"Quartier De Castel Maou - Chemin d'Aouredou -,...",42.71383156,1.83920681,Ariège day 5


In [57]:
city_and_hotel_info_df

,city_name,place_id,lat,lon,hotel_name,hotel_url,score,description,location,hotel_latitude,hotel_longtitude,day_in_city
0,Ariège,297389050.0,42.945537,1.406554,Lagrange Vacances Les Chalets d’Ax,https://www.booking.com/hotel/fr/residence-les...,7.6,Lagrange Vacances Les Chalets d’Ax is located ...,"Quartier De Castel Maou - Chemin d'Aouredou -,...",42.71383156,1.83920681,Ariège day 1
0,Ariège,297389050.0,42.945537,1.406554,Lagrange Vacances Les Chalets d’Ax,https://www.booking.com/hotel/fr/residence-les...,7.6,Lagrange Vacances Les Chalets d’Ax is located ...,"Quartier De Castel Maou - Chemin d'Aouredou -,...",42.71383156,1.83920681,Ariège day 2
0,Ariège,297389050.0,42.945537,1.406554,Lagrange Vacances Les Chalets d’Ax,https://www.booking.com/hotel/fr/residence-les...,7.6,Lagrange Vacances Les Chalets d’Ax is located ...,"Quartier De Castel Maou - Chemin d'Aouredou -,...",42.71383156,1.83920681,Ariège day 3
0,Ariège,297389050.0,42.945537,1.406554,Lagrange Vacances Les Chalets d’Ax,https://www.booking.com/hotel/fr/residence-les...,7.6,Lagrange Vacances Les Chalets d’Ax is located ...,"Quartier De Castel Maou - Chemin d'Aouredou -,...",42.71383156,1.83920681,Ariège day 4
0,Ariège,297389050.0,42.945537,1.406554,Lagrange Vacances Les Chalets d’Ax,https://www.booking.com/hotel/fr/residence-les...,7.6,Lagrange Vacances Les Chalets d’Ax is located ...,"Quartier De Castel Maou - Chemin d'Aouredou -,...",42.71383156,1.83920681,Ariège day 5
...,...,...,...,...,...,...,...,...,...,...,...,...
999,Toulouse,298222566.0,43.604462,1.444247,Hotel Albert 1er,https://www.booking.com/hotel/fr/hotelalbertto...,8.3,"Located in the heart of Toulouse, the Hotel Al...","8 rue Rivals, 31000 Toulouse, France",43.60603220,1.44491941,Toulouse day 3
999,Toulouse,298222566.0,43.604462,1.444247,Hotel Albert 1er,https://www.booking.com/hotel/fr/hotelalbertto...,8.3,"Located in the heart of Toulouse, the Hotel Al...","8 rue Rivals, 31000 Toulouse, France",43.60603220,1.44491941,Toulouse day 4
999,Toulouse,298222566.0,43.604462,1.444247,Hotel Albert 1er,https://www.booking.com/hotel/fr/hotelalbertto...,8.3,"Located in the heart of Toulouse, the Hotel Al...","8 rue Rivals, 31000 Toulouse, France",43.60603220,1.44491941,Toulouse day 5
999,Toulouse,298222566.0,43.604462,1.444247,Hotel Albert 1er,https://www.booking.com/hotel/fr/hotelalbertto...,8.3,"Located in the heart of Toulouse, the Hotel Al...","8 rue Rivals, 31000 Toulouse, France",43.60603220,1.44491941,Toulouse day 6


In [58]:
len(city_and_hotel_info_df)

7000

In [59]:
# Loading .csv file with weather data from S3 bucket
url = 'https://kayak-booking-bucket-12-12-2022.s3.eu-west-3.amazonaws.com/weather_forecast.csv'
weather_df = pd.read_csv(url)
weather_df.head(5)

,Unnamed: 0,dt,sunrise,sunset,pressure,humidity,dew_point,wind_speed,wind_deg,wind_gust,...,temp_eve,temp_morn,feels_like_day,feels_like_night,feels_like_eve,feels_like_morn,weather_id,weather_main,weather_description,city_name
0,0,2023-02-20 13:00:00,2023-02-20 08:05:53,2023-02-20 18:34:05,1028,71,5.56,2.06,208,2.25,...,7.46,4.75,9.80,6.81,7.46,3.15,800.0,Clear,clear sky,Mont Saint-Michel
1,1,2023-02-21 13:00:00,2023-02-21 08:04:04,2023-02-21 18:35:42,1018,64,5.72,2.67,139,4.01,...,9.88,5.43,11.48,7.86,9.43,3.54,804.0,Clouds,overcast clouds,Mont Saint-Michel
2,2,2023-02-22 13:00:00,2023-02-22 08:02:13,2023-02-22 18:37:18,1015,88,5.93,6.78,318,10.43,...,6.31,8.70,4.68,2.21,2.76,7.03,500.0,Rain,light rain,Mont Saint-Michel
3,3,2023-02-23 13:00:00,2023-02-23 08:00:22,2023-02-23 18:38:55,1018,76,3.98,7.16,31,12.58,...,6.88,5.10,4.90,2.04,2.94,4.15,500.0,Rain,light rain,Mont Saint-Michel
4,4,2023-02-24 13:00:00,2023-02-24 07:58:29,2023-02-24 18:40:31,1015,67,3.02,6.72,29,12.65,...,6.93,2.54,6.21,1.99,4.91,-0.90,803.0,Clouds,broken clouds,Mont Saint-Michel


In [60]:
len(weather_df)

245

In [61]:
weather_df.head()

,Unnamed: 0,dt,sunrise,sunset,pressure,humidity,dew_point,wind_speed,wind_deg,wind_gust,...,temp_eve,temp_morn,feels_like_day,feels_like_night,feels_like_eve,feels_like_morn,weather_id,weather_main,weather_description,city_name
0,0,2023-02-20 13:00:00,2023-02-20 08:05:53,2023-02-20 18:34:05,1028,71,5.56,2.06,208,2.25,...,7.46,4.75,9.80,6.81,7.46,3.15,800.0,Clear,clear sky,Mont Saint-Michel
1,1,2023-02-21 13:00:00,2023-02-21 08:04:04,2023-02-21 18:35:42,1018,64,5.72,2.67,139,4.01,...,9.88,5.43,11.48,7.86,9.43,3.54,804.0,Clouds,overcast clouds,Mont Saint-Michel
2,2,2023-02-22 13:00:00,2023-02-22 08:02:13,2023-02-22 18:37:18,1015,88,5.93,6.78,318,10.43,...,6.31,8.70,4.68,2.21,2.76,7.03,500.0,Rain,light rain,Mont Saint-Michel
3,3,2023-02-23 13:00:00,2023-02-23 08:00:22,2023-02-23 18:38:55,1018,76,3.98,7.16,31,12.58,...,6.88,5.10,4.90,2.04,2.94,4.15,500.0,Rain,light rain,Mont Saint-Michel
4,4,2023-02-24 13:00:00,2023-02-24 07:58:29,2023-02-24 18:40:31,1015,67,3.02,6.72,29,12.65,...,6.93,2.54,6.21,1.99,4.91,-0.90,803.0,Clouds,broken clouds,Mont Saint-Michel


In [62]:
nb_cities = weather_df["city_name"].nunique()
days = [1, 2, 3, 4, 5, 6, 7]
day_numbers = days * nb_cities
weather_df["day_in_city"] = day_numbers
weather_df["day_in_city"] = weather_df['city_name'] + " day "+ weather_df["day_in_city"].astype(str)


In [63]:
weather_df.tail(15)

,Unnamed: 0,dt,sunrise,sunset,pressure,humidity,dew_point,wind_speed,wind_deg,wind_gust,...,temp_morn,feels_like_day,feels_like_night,feels_like_eve,feels_like_morn,weather_id,weather_main,weather_description,city_name,day_in_city
230,230,2023-02-26 13:00:00,2023-02-26 07:48:33,2023-02-26 18:50:09,1017,61,-0.52,5.99,45,9.68,...,1.62,4.23,0.91,3.59,-1.82,801.0,Clouds,few clouds,Biarritz,Biarritz day 7
231,231,2023-02-20 13:00:00,2023-02-20 07:57:43,2023-02-20 18:41:57,1023,58,8.92,3.61,84,6.46,...,8.97,16.89,12.42,18.00,7.49,804.0,Clouds,overcast clouds,Bayonne,Bayonne day 1
232,232,2023-02-21 13:00:00,2023-02-21 07:56:11,2023-02-21 18:43:17,1020,64,8.30,3.83,273,7.90,...,11.66,14.48,11.73,11.60,10.71,804.0,Clouds,overcast clouds,Bayonne,Bayonne day 2
233,233,2023-02-22 13:00:00,2023-02-22 07:54:38,2023-02-22 18:44:36,1020,89,10.64,8.89,338,12.12,...,11.39,12.25,5.98,10.56,11.01,502.0,Rain,heavy intensity rain,Bayonne,Bayonne day 3
234,234,2023-02-23 13:00:00,2023-02-23 07:53:04,2023-02-23 18:45:54,1016,83,4.51,7.34,318,12.07,...,7.74,7.43,5.45,6.83,5.49,502.0,Rain,heavy intensity rain,Bayonne,Bayonne day 4
235,235,2023-02-24 13:00:00,2023-02-24 07:51:29,2023-02-24 18:47:13,1012,80,4.42,3.16,220,4.68,...,4.45,5.84,3.06,7.48,4.45,500.0,Rain,light rain,Bayonne,Bayonne day 5
236,236,2023-02-25 13:00:00,2023-02-25 07:49:53,2023-02-25 18:48:31,1010,89,3.79,3.70,39,7.52,...,3.58,5.66,3.83,7.28,0.82,500.0,Rain,light rain,Bayonne,Bayonne day 6
237,237,2023-02-26 13:00:00,2023-02-26 07:48:16,2023-02-26 18:49:49,1016,58,-0.62,5.17,45,10.54,...,1.10,5.35,0.84,3.59,-1.67,801.0,Clouds,few clouds,Bayonne,Bayonne day 7
238,238,2023-02-20 13:00:00,2023-02-20 08:00:24,2023-02-20 18:36:42,1026,67,6.39,4.79,59,6.65,...,6.30,11.54,7.07,12.69,3.52,801.0,Clouds,few clouds,La Rochelle,La Rochelle day 1
239,239,2023-02-21 13:00:00,2023-02-21 07:58:43,2023-02-21 18:38:10,1017,73,7.61,4.75,231,8.03,...,8.70,11.58,7.61,9.97,6.38,804.0,Clouds,overcast clouds,La Rochelle,La Rochelle day 2


In [64]:
weather_df.columns

Index(['Unnamed: 0', 'dt', 'sunrise', 'sunset', 'pressure', 'humidity',
       'dew_point', 'wind_speed', 'wind_deg', 'wind_gust', 'clouds', 'pop',
       'uvi', 'rain', 'snow', 'temp_day', 'temp_min', 'temp_max', 'temp_night',
       'temp_eve', 'temp_morn', 'feels_like_day', 'feels_like_night',
       'feels_like_eve', 'feels_like_morn', 'weather_id', 'weather_main',
       'weather_description', 'city_name', 'day_in_city'],
      dtype='object')

We change the column order to be able to merge dataframes easily.
We will keep 'city_name' column in both dataframes in order to check later if the merge has been done correctly.

In [65]:
weather_df = weather_df[['day_in_city', 'city_name', 'dt', 'sunrise', 'sunset', 'pressure', 'humidity',
       'dew_point', 'wind_speed', 'wind_deg', 'wind_gust', 'clouds', 'pop',
       'uvi', 'rain', 'snow', 'temp_day', 'temp_min', 'temp_max', 'temp_night',
       'temp_eve', 'temp_morn', 'feels_like_day', 'feels_like_night',
       'feels_like_eve', 'feels_like_morn', 'weather_id', 'weather_main',
       'weather_description']]

In [66]:
weather_df.head()

,day_in_city,city_name,dt,sunrise,sunset,pressure,humidity,dew_point,wind_speed,wind_deg,...,temp_night,temp_eve,temp_morn,feels_like_day,feels_like_night,feels_like_eve,feels_like_morn,weather_id,weather_main,weather_description
0,Mont Saint-Michel day 1,Mont Saint-Michel,2023-02-20 13:00:00,2023-02-20 08:05:53,2023-02-20 18:34:05,1028,71,5.56,2.06,208,...,7.40,7.46,4.75,9.80,6.81,7.46,3.15,800.0,Clear,clear sky
1,Mont Saint-Michel day 2,Mont Saint-Michel,2023-02-21 13:00:00,2023-02-21 08:04:04,2023-02-21 18:35:42,1018,64,5.72,2.67,139,...,8.66,9.88,5.43,11.48,7.86,9.43,3.54,804.0,Clouds,overcast clouds
2,Mont Saint-Michel day 3,Mont Saint-Michel,2023-02-22 13:00:00,2023-02-22 08:02:13,2023-02-22 18:37:18,1015,88,5.93,6.78,318,...,4.48,6.31,8.70,4.68,2.21,2.76,7.03,500.0,Rain,light rain
3,Mont Saint-Michel day 4,Mont Saint-Michel,2023-02-23 13:00:00,2023-02-23 08:00:22,2023-02-23 18:38:55,1018,76,3.98,7.16,31,...,6.18,6.88,5.10,4.90,2.04,2.94,4.15,500.0,Rain,light rain
4,Mont Saint-Michel day 5,Mont Saint-Michel,2023-02-24 13:00:00,2023-02-24 07:58:29,2023-02-24 18:40:31,1015,67,3.02,6.72,29,...,4.61,6.93,2.54,6.21,1.99,4.91,-0.90,803.0,Clouds,broken clouds


In [67]:
kayak_df = city_and_hotel_info_df.merge(weather_df, how="left")

In [68]:
len(kayak_df)

7000

In [69]:
kayak_df.tail(15)

,city_name,place_id,lat,lon,hotel_name,hotel_url,score,description,location,hotel_latitude,...,temp_night,temp_eve,temp_morn,feels_like_day,feels_like_night,feels_like_eve,feels_like_morn,weather_id,weather_main,weather_description
6985,Toulouse,298222566.0,43.604462,1.444247,T2 COSY - CAPITOLE - CALME - Wi-Fi - NETFLIX,https://www.booking.com/hotel/fr/studio-cosy-h...,8.7,"Situated in the centre of Toulouse, 3.7 km fro...","1er étage 44 Rue Saint-Rome, 31000 Toulouse, F...",43.60330820,...,2.11,4.90,-0.17,2.85,-2.10,1.67,-4.49,801.0,Clouds,few clouds
6986,Toulouse,298222566.0,43.604462,1.444247,Modern flat 50m from the Capitole - Toulouse -...,https://www.booking.com/hotel/fr/modern-flat-5...,8.3,"Set in the centre of Toulouse, 3.3 km from Zén...","43 rue Saint-Rome, 31000 Toulouse, France",43.60333730,...,9.74,17.64,6.98,14.70,7.29,16.69,6.98,802.0,Clouds,scattered clouds
6987,Toulouse,298222566.0,43.604462,1.444247,Modern flat 50m from the Capitole - Toulouse -...,https://www.booking.com/hotel/fr/modern-flat-5...,8.3,"Set in the centre of Toulouse, 3.3 km from Zén...","43 rue Saint-Rome, 31000 Toulouse, France",43.60333730,...,11.44,13.53,8.01,14.42,10.99,12.77,5.30,500.0,Rain,light rain
6988,Toulouse,298222566.0,43.604462,1.444247,Modern flat 50m from the Capitole - Toulouse -...,https://www.booking.com/hotel/fr/modern-flat-5...,8.3,"Set in the centre of Toulouse, 3.3 km from Zén...","43 rue Saint-Rome, 31000 Toulouse, France",43.60333730,...,10.21,11.06,9.59,11.86,9.69,10.55,9.01,500.0,Rain,light rain
6989,Toulouse,298222566.0,43.604462,1.444247,Modern flat 50m from the Capitole - Toulouse -...,https://www.booking.com/hotel/fr/modern-flat-5...,8.3,"Set in the centre of Toulouse, 3.3 km from Zén...","43 rue Saint-Rome, 31000 Toulouse, France",43.60333730,...,5.70,6.19,5.52,5.37,4.73,4.79,1.82,501.0,Rain,moderate rain
6990,Toulouse,298222566.0,43.604462,1.444247,Modern flat 50m from the Capitole - Toulouse -...,https://www.booking.com/hotel/fr/modern-flat-5...,8.3,"Set in the centre of Toulouse, 3.3 km from Zén...","43 rue Saint-Rome, 31000 Toulouse, France",43.60333730,...,5.62,5.78,4.41,3.73,4.65,5.78,2.70,500.0,Rain,light rain
6991,Toulouse,298222566.0,43.604462,1.444247,Modern flat 50m from the Capitole - Toulouse -...,https://www.booking.com/hotel/fr/modern-flat-5...,8.3,"Set in the centre of Toulouse, 3.3 km from Zén...","43 rue Saint-Rome, 31000 Toulouse, France",43.60333730,...,5.56,6.21,5.12,3.80,3.43,3.20,3.13,500.0,Rain,light rain
6992,Toulouse,298222566.0,43.604462,1.444247,Modern flat 50m from the Capitole - Toulouse -...,https://www.booking.com/hotel/fr/modern-flat-5...,8.3,"Set in the centre of Toulouse, 3.3 km from Zén...","43 rue Saint-Rome, 31000 Toulouse, France",43.60333730,...,2.11,4.90,-0.17,2.85,-2.10,1.67,-4.49,801.0,Clouds,few clouds
6993,Toulouse,298222566.0,43.604462,1.444247,Hotel Albert 1er,https://www.booking.com/hotel/fr/hotelalbertto...,8.3,"Located in the heart of Toulouse, the Hotel Al...","8 rue Rivals, 31000 Toulouse, France",43.60603220,...,9.74,17.64,6.98,14.70,7.29,16.69,6.98,802.0,Clouds,scattered clouds
6994,Toulouse,298222566.0,43.604462,1.444247,Hotel Albert 1er,https://www.booking.com/hotel/fr/hotelalbertto...,8.3,"Located in the heart of Toulouse, the Hotel Al...","8 rue Rivals, 31000 Toulouse, France",43.60603220,...,11.44,13.53,8.01,14.42,10.99,12.77,5.30,500.0,Rain,light rain


In [70]:
kayak_df.columns

Index(['city_name', 'place_id', 'lat', 'lon', 'hotel_name', 'hotel_url',
       'score', 'description', 'location', 'hotel_latitude',
       'hotel_longtitude', 'day_in_city', 'dt', 'sunrise', 'sunset',
       'pressure', 'humidity', 'dew_point', 'wind_speed', 'wind_deg',
       'wind_gust', 'clouds', 'pop', 'uvi', 'rain', 'snow', 'temp_day',
       'temp_min', 'temp_max', 'temp_night', 'temp_eve', 'temp_morn',
       'feels_like_day', 'feels_like_night', 'feels_like_eve',
       'feels_like_morn', 'weather_id', 'weather_main', 'weather_description'],
      dtype='object')

Now we have the dataframe with all the necessary information, but some adjustments have to be made.

We'll rename some of the columns for clarity:

In [71]:
kayak_df.rename(columns = {'place_id':'city_id'}, inplace = True)
kayak_df.rename(columns = {'lat':'city_latitude'}, inplace = True)
kayak_df.rename(columns = {'lon':'city_longtitude'}, inplace = True)
kayak_df.rename(columns = {'dt':'date'}, inplace = True)

# create a function for renaming columns


Values in some columns are still in string format while they should be in float or datetime format:

In [72]:
kayak_df.dtypes

city_name               object
city_id                float64
city_latitude          float64
city_longtitude        float64
hotel_name              object
hotel_url               object
score                   object
description             object
location                object
hotel_latitude          object
hotel_longtitude        object
day_in_city             object
date                    object
sunrise                 object
sunset                  object
pressure               float64
humidity               float64
dew_point              float64
wind_speed             float64
wind_deg               float64
wind_gust              float64
clouds                 float64
pop                    float64
uvi                    float64
rain                   float64
snow                   float64
temp_day               float64
temp_min               float64
temp_max               float64
temp_night             float64
temp_eve               float64
temp_morn              float64
feels_li

We'll convert the values columns "date", "sunrise", "sunset" to datetime format.

In [73]:
columns_to_convert = ["date", "sunrise", "sunset"]
for column in columns_to_convert: 
    kayak_df[column] = pd.to_datetime(kayak_df[column], format='%Y-%m-%d %H:%M:%S')

Checking which are the columns that still contain values in string ("object") format

In [75]:
(kayak_df.select_dtypes(include=['object'])).columns

Index(['city_name', 'hotel_name', 'hotel_url', 'score', 'description',
       'location', 'hotel_latitude', 'hotel_longtitude', 'day_in_city',
       'weather_main', 'weather_description'],
      dtype='object')

In [76]:
columns_to_convert = ['score', 'hotel_latitude', 'hotel_longtitude']
for column in columns_to_convert: 
    kayak_df[column] = kayak_df[column].astype(float)

# Checking the value type for dataframe columns:
print(kayak_df.dtypes)

city_name                      object
city_id                       float64
city_latitude                 float64
city_longtitude               float64
hotel_name                     object
hotel_url                      object
score                         float64
description                    object
location                       object
hotel_latitude                float64
hotel_longtitude              float64
day_in_city                    object
date                   datetime64[ns]
sunrise                datetime64[ns]
sunset                 datetime64[ns]
pressure                      float64
humidity                      float64
dew_point                     float64
wind_speed                    float64
wind_deg                      float64
wind_gust                     float64
clouds                        float64
pop                           float64
uvi                           float64
rain                          float64
snow                          float64
temp_day    

In [77]:
kayak_df["date"][29]

Timestamp('2023-02-21 13:00:00')

In [78]:
(kayak_df["date"][29]).day_name()

'Tuesday'

In [79]:
kayak_df["day_of_week"] = kayak_df["date"].apply(lambda x: x.day_name())

In [82]:
print(kayak_df.columns)

# Rearranging the order of columns for clarity
kayak_df = kayak_df[['city_name', 'city_id', 'city_latitude', 'city_longtitude',
       'hotel_name', 'hotel_url', 'score', 'description', 'location',
       'hotel_latitude', 'hotel_longtitude', 'date', 'day_of_week','day_in_city','sunrise',
       'sunset', 'weather_main', 'weather_description', 'weather_id','pressure', 
       'humidity', 'dew_point', 'wind_speed', 'wind_deg',
       'wind_gust', 'clouds', 'pop', 'uvi', 'rain', 'snow', 'temp_day',
       'temp_min', 'temp_max', 'temp_night', 'temp_eve', 'temp_morn',
       'feels_like_day', 'feels_like_night', 'feels_like_eve','feels_like_morn']]

In [84]:
kayak_df.to_csv("kayak.csv", index=False)

Saving the .csv file to S3 bucket

In [94]:
# (The cell is commented to avoid unnecessary rewriting).
"""# Access key for user with access to write in S3 bucket
S3_ACCESS_KEY_ID =  os.getenv("S3_ACCESS_KEY_ID")
# Secret key for user with access to write in S3 bucket 
S3_SECRET_ACCESS_KEY =  os.getenv("S3_SECRET_ACCESS_KEY")

# Writing the .csv file to bucket S3
session = boto3.Session(aws_access_key_id=S3_ACCESS_KEY_ID, 
                      aws_secret_access_key=S3_SECRET_ACCESS_KEY)
s3 = session.resource("s3")
bucket = s3.Bucket("kayak-booking-bucket-12-12-2022") 
bucket.upload_file("kayak.csv", Key="kayak.csv")"""

'# Access key for user with access to write in S3 bucket\nS3_ACCESS_KEY_ID =  os.getenv("S3_ACCESS_KEY_ID")\n# Secret key for user with access to write in S3 bucket \nS3_SECRET_ACCESS_KEY =  os.getenv("S3_SECRET_ACCESS_KEY")\n\n# Writing the .csv file to bucket S3\nsession = boto3.Session(aws_access_key_id=S3_ACCESS_KEY_ID, \n                      aws_secret_access_key=S3_SECRET_ACCESS_KEY)\ns3 = session.resource("s3")\nbucket = s3.Bucket("kayak-booking-bucket-12-12-2022") \nbucket.upload_file("kayak.csv", Key="kayak.csv")'